In [1]:
import os

In [2]:
%pwd

'/Users/azizamed/Downloads/week10/Automated-Storyboard-Synthesis-for-Digital-Advertising/notebooks/data_pipeline'

In [3]:
os.chdir('../../')

In [4]:
%pwd

'/Users/azizamed/Downloads/week10/Automated-Storyboard-Synthesis-for-Digital-Advertising'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from data_ingest.constants import *
from data_ingest.utils.common import read_yaml, create_directories

ModuleNotFoundError: No module named 'data_ingest'

In [15]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])



    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [16]:
import os
import zipfile
import gdown
from data_ingest import logger
from data_ingest.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [18]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-01 04:43:42,359: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-01 04:43:42,362: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-01 04:43:42,363: INFO: common: created directory at: artifacts]
[2024-02-01 04:43:42,364: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-01 04:43:42,365: INFO: 1758446011: Downloading data from https://drive.google.com/file/d/1F7mxKqGxTn_y_IJT_o03Ub4DQr04buSB/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1F7mxKqGxTn_y_IJT_o03Ub4DQr04buSB
To: /Users/azizamed/Downloads/week7/AmharicAI-AdGen/artifacts/data_ingestion/data.zip
100%|██████████| 28.6k/28.6k [00:00<00:00, 147kB/s]


[2024-02-01 04:43:45,999: INFO: 1758446011: Downloaded data from https://drive.google.com/file/d/1F7mxKqGxTn_y_IJT_o03Ub4DQr04buSB/view?usp=sharing into file artifacts/data_ingestion/data.zip]
